In [22]:
import joblib
import numpy as np
import pandas as pd

X_train = joblib.load("../data/mutation/X_train.pkl")
y_train = joblib.load("../data/mutation/y_train.pkl")
X_test = joblib.load("../data/mutation/X_test.pkl")
y_test = joblib.load("../data/mutation/y_test.pkl")
LASSO_model = joblib.load("../models/LASSO_with_mutation")


In [23]:
import numpy as np
import pandas as pd
import time
import joblib
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, precision_score, recall_score
from xgboost import XGBClassifier
from itertools import product

def run_XGBoost_from_LASSO_with_grid(
    lasso_model_path="../models/LASSO_with_mutation",
    X=None, y=None, top_k=500, seed=42,
    param_grid=None, n_splits=3
):
    """
    Load LASSO model, select top_k features, and run custom CV grid search with XGBoost.
    No early stopping (for full compatibility).
    """
    # Load LASSO
    model = joblib.load(lasso_model_path)
    coefs = model.named_steps["clf"].coef_.ravel()
    feature_names = X.columns
    abs_coefs = np.abs(coefs)

    # Select top_k features
    top_k = min(top_k, X.shape[1])
    top_idx = np.argsort(abs_coefs)[-top_k:]
    selected_features = feature_names[top_idx]
    X_sub = X[selected_features].copy()

    print(f"Selected top {top_k} features from LASSO model")

    # Imbalance handling
    y_arr = y.astype(int) if hasattr(y, "astype") else np.asarray(y).astype(int)
    neg, pos = np.bincount(y_arr)
    spw = neg / pos if pos > 0 else 1.0
    print(f"⚖️ scale_pos_weight set to {spw:.2f} (neg={neg}, pos={pos})")

    # Default param grid
    if param_grid is None:
        param_grid = {
            "n_estimators": [200, 400],
            "max_depth": [3, 5],
            "learning_rate": [0.01, 0.1],
            "subsample": [0.8, 1.0],
            "colsample_bytree": [0.5, 0.8],
        }

    keys = list(param_grid.keys())
    grid_combos = [dict(zip(keys, v)) for v in product(*param_grid.values())]

    print(f"🔍 Running grid search over {len(grid_combos)} parameter sets")

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    results = []

    for combo_i, params in enumerate(grid_combos, 1):
        print(f"\n=== Grid {combo_i}/{len(grid_combos)}: {params} ===")

        fold_metrics = []
        for fold, (train_idx, val_idx) in enumerate(skf.split(X_sub, y_arr), 1):
            start = time.time()
            X_train, X_val = X_sub.iloc[train_idx], X_sub.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx].astype(int), y.iloc[val_idx].astype(int)

            clf_kwargs = {
                "objective": "binary:logistic",
                "eval_metric": "auc",
                "random_state": seed,
                "n_jobs": -1,
                "tree_method": "hist",
                "scale_pos_weight": spw,
                "use_label_encoder": False,
                **params
            }
            clf = XGBClassifier(**clf_kwargs)

            # Plain fit
            clf.fit(X_train, y_train, verbose=False)

            # Predictions
            y_pred_proba = clf.predict_proba(X_val)[:, 1]
            y_pred = (y_pred_proba >= 0.5).astype(int)

            auc = roc_auc_score(y_val, y_pred_proba)
            prec = precision_score(y_val, y_pred, zero_division=0)
            rec = recall_score(y_val, y_pred, zero_division=0)

            elapsed = time.time() - start
            print(f"✅ Fold {fold}/{n_splits} | AUC={auc:.3f}, Prec={prec:.3f}, Rec={rec:.3f} | Time {elapsed:.1f}s")

            fold_metrics.append((auc, prec, rec))

        mean_auc = np.mean([m[0] for m in fold_metrics])
        mean_prec = np.mean([m[1] for m in fold_metrics])
        mean_rec = np.mean([m[2] for m in fold_metrics])

        results.append({
            "params": params,
            "auc": mean_auc,
            "precision": mean_prec,
            "recall": mean_rec
        })

        print(f"📊 Combo {combo_i} mean: AUC={mean_auc:.3f}, Prec={mean_prec:.3f}, Rec={mean_rec:.3f}")

    results_df = pd.DataFrame(results)
    best_idx = results_df["auc"].idxmax()
    best_params = results_df.iloc[best_idx]["params"]

    print("\n🏆 Best params:", best_params)
    print("🏆 Best CV AUC:", results_df.iloc[best_idx]['auc'])

    return results_df, selected_features


In [24]:
fold_results, selected_features = run_XGBoost_from_LASSO_with_grid(
    lasso_model_path="../models/LASSO_with_mutation",
    X=X_train,
    y=y_train,
    top_k=500, seed=42)

Selected top 500 features from LASSO model
⚖️ scale_pos_weight set to 4.47 (neg=295, pos=66)
🔍 Running grid search over 32 parameter sets

=== Grid 1/32: {'n_estimators': 200, 'max_depth': 3, 'learning_rate': 0.01, 'subsample': 0.8, 'colsample_bytree': 0.5} ===


c:\Users\gench\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [13:32:17] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


✅ Fold 1/3 | AUC=0.738, Prec=0.467, Rec=0.318 | Time 3.2s


c:\Users\gench\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [13:32:20] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


✅ Fold 2/3 | AUC=0.704, Prec=0.467, Rec=0.318 | Time 2.5s


...

✅ Fold 1/3 | AUC=0.770, Prec=0.500, Rec=0.273 | Time 4.3s


c:\Users\gench\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [13:39:08] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


✅ Fold 2/3 | AUC=0.782, Prec=0.667, Rec=0.364 | Time 3.7s


c:\Users\gench\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [13:39:12] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


✅ Fold 3/3 | AUC=0.746, Prec=0.600, Rec=0.409 | Time 3.7s
📊 Combo 32 mean: AUC=0.766, Prec=0.589, Rec=0.348

🏆 Best params: {'n_estimators': 400, 'max_depth': 3, 'learning_rate': 0.1, 'subsample': 0.8, 'colsample_bytree': 0.8}
🏆 Best CV AUC: 0.7776934465246154


In [25]:
import xgboost

print(xgboost.__version__)


3.0.0


In [ ]:
# Training Data Evaluation Metrics

# Predictions (discrete)
y_train_pred = best_model.predict(X_train)

print("Training Data Evaluation Metrics:")

# Confusion matrix
cm_train = confusion_matrix(y_train, y_train_pred)
tn, fp, fn, tp = cm_train.ravel()
print(f"True Positives (TP): {tp}")
print(f"False Positives (FP): {fp}")
print(f"True Negatives (TN): {tn}")
print(f"False Negatives (FN): {fn}")

# Precision & Recall
precision = precision_score(y_train, y_train_pred)
recall = recall_score(y_train, y_train_pred)
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")

# Continuous scores for AUROC
if hasattr(best_model, "predict_proba"):
    y_scores = best_model.predict_proba(X_train)[:, 1]
elif hasattr(best_model, "decision_function"):
    y_scores = best_model.decision_function(X_train)
else:
    y_scores = y_train_pred  # fallback (not ideal)

# AUROC
roc_auc = roc_auc_score(y_train, y_scores)
print(f"AUC-ROC Score: {roc_auc:.4f}")

# ROC curve
fpr, tpr, _ = roc_curve(y_train, y_scores)
plt.figure(figsize=(6, 6))
plt.plot(fpr, tpr, linestyle='-', label=f'ROC Curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')  # Random classifier line
plt.xlabel("False Positive Rate (FPR)")
plt.ylabel("True Positive Rate (TPR)")
plt.title("ROC Curve on Training Data")
plt.legend()
plt.show()